# Portafolios de Inversión
## Tarea 6

Presenta : 

- José Armando Melchor Soto

- Paula Ines Pelayo Morales 

---
### Librerías 



In [271]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize 

---

#### Indicaciones 

- Punto a) Implementen tres funciones que optimicen portafolios utilizando: mínima semivarianza, ratio omega y semivarianza objetivo, empleando la función `scipy.optimize.minimize`. (40%)
 

- Punto b) Construyan un portafolio compuesto por 5 activos financieros y optimícenlo con los tres métodos. Utilicen las funciones desarrolladas en el punto a) para realizar la optimización. Recuerden que, para el portafolio con semivarianza objetivo, deberán seleccionar un benchmark adecuado y consistente con los activos elegidos. Como respuesta a este punto, se esperan las ponderaciones eficientes y el valor óptimo de la función objetivo para cada método de Asset Allocation. (40%)
 

- Punto c) Escriban una breve conclusión sobre las ventajas y desventajas que observan entre los métodos TMP y TPMP. (20%)

---

#### Funciones 

#### Minimizar Semi-Varianza

**Minimizar semivarianza**

$$min_w \hspace{0.5cm} \sigma_d^2 = w^T S w$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$



##### Mínima Semi-Varianza  (Scipy Library)

In [272]:
def scipy_semivar(rets: pd.DataFrame, corr: pd.DataFrame):

    rets_below_zero=rets[rets < 0].fillna(0)
    
    downside_risk=rets_below_zero.std()

    downside_risk=np.array(downside_risk)
    
    semivar_matrix = downside_risk.reshape(len(rets.columns), 1) @ downside_risk.reshape(1, len(rets.columns)) * corr

    n = len(rets.columns)
    w_inicial = np.ones(n) / n
    bounds = [(0, 1)] * n
    tol = 1e-9

    semivar = lambda w: (w.T @ semivar_matrix @ w)

    constraints = {'fun': lambda w: np.sum(w) - 1, 'type': 'eq'}

    minima_semi_var = minimize(
        fun=semivar,
        x0=w_inicial,
        bounds=bounds,
        constraints=constraints,
        tol=tol)

    w_semi_var = minima_semi_var.x
    return dict(zip(rets.columns, w_semi_var))





---

#### Ratio Omega 

**Calcular Omega del portafolio y maximizarla cambiando ponderaciones.**

$$max_w \hspace{0.5cm} \Omega_p = \sum_{i=1}^{n} \Omega_i * w_i$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$

##### Ratio Omega (Scipy Library)

In [273]:
def scipy_omega(rets: pd.DataFrame):
    n = len(rets.columns)
    w_inicial = np.ones(n) / n
    bounds = [(0, 3)] * n
    tol = 1e-9

    rets_below_zero = rets[rets < 0].fillna(0)
    rets_above_zero = rets[rets > 0].fillna(0)
    upside_risk=rets_above_zero.std()
    downside_risk = rets_below_zero.std()

    omega_ind = upside_risk / downside_risk

    Omega_neg = lambda w: -np.sum(w * omega_ind)

    constraints = {'fun': lambda w: np.sum(w) - 1, 'type': 'eq'}

    result = minimize(
        fun=Omega_neg,
        x0=w_inicial,
        bounds=bounds,
        constraints=constraints,
        tol=tol
    )

    w_omega = result.x
    return dict(zip(rets.columns, w_omega))


---

#### Semi-Varianza Objetivo

##### Semi-Varianza Objetivo (Scipy)

In [274]:
def objetivo (rets:pd.DataFrame , rets_b : pd.DataFrame ,corr:pd.DataFrame):

    diffs=rets - rets_b.values
    rends_below_bench = diffs[diffs < 0].fillna(0)
    target_downside_risk=np.array(rends_below_bench.std())
    target_semivarmatrix= corr * (target_downside_risk.reshape(len(rets.columns), 1) @ target_downside_risk.reshape(1, len(rets.columns)))

    objectivo = lambda w: (w.T @ target_semivarmatrix @ w)

    n = len(rets.keys())
    w_inicial = np.ones(n)/n
    bounds = [(0,1)]*n
    tol = 1e-9
    rest = lambda w: np.sum(w) - 1
    
    obj = minimize(
    fun=objectivo,
    x0=w_inicial,
    bounds=bounds,
    constraints={'fun': rest, 'type': 'eq'},
    tol=tol)

    w_objetivo = obj.x

    return dict(zip(rets.columns, w_objetivo))

---

#### Construcción del Portafolio 

##### Activos 

##### Sector económico : Salud 

- Eli Lilly and Company (LLY)

- Intuitive Surgical, Inc. (ISRG)

- AbbVie Inc. (ABBV)

- Johnson & Johnson (JNJ)

- Merck & Co., Inc. (MRK)


##### Benchmark

##### Sector económico : Consumo básico 

- Vanguard Consumer Staples Index Fund ETF Shares (VDC)

---

#### Importación de los datos

##### Activos 

In [275]:
prices = yf.download(['LLY', 'ISRG', 'ABBV', 'JNJ', 'MRK'], start='2020-01-01', end='2025-06-19')['Close']
prices.head()

[*********************100%***********************]  5 of 5 completed


Ticker,ABBV,ISRG,JNJ,LLY,MRK
Date,,,,,
2020-01-02,70.934235,199.086670,124.983665,123.449234,73.971642
2020-01-03,70.260933,197.779999,123.536652,123.038414,73.336731
2020-01-06,70.815422,198.586670,123.382530,123.495941,73.650177
2020-01-07,70.411438,194.266663,124.135994,123.729378,71.689171
2020-01-08,70.910477,193.926666,124.118896,124.849876,71.206947


##### Benchmarck 

In [276]:
benchmark = yf.download('VDC', start='2020-01-01', end='2025-06-19')['Close']
benchmark.head()

[*********************100%***********************]  1 of 1 completed


Ticker,VDC
Date,
2020-01-02,140.328201
2020-01-03,140.082382
2020-01-06,140.293060
2020-01-07,139.222198
2020-01-08,139.775177


---

#### Construcción del Portafolio 

##### Rendimientos Diarios

In [277]:
rets = prices.pct_change().dropna()
rets.head()

Ticker,ABBV,ISRG,JNJ,LLY,MRK
Date,,,,,
2020-01-03,-0.009492,-0.006563,-0.011578,-0.003328,-0.008583
2020-01-06,0.007892,0.004079,-0.001248,0.003719,0.004274
2020-01-07,-0.005705,-0.021754,0.006107,0.001890,-0.026626
2020-01-08,0.007087,-0.001750,-0.000138,0.009056,-0.006727
2020-01-09,0.007708,0.006893,0.002966,0.016528,0.008804


##### Rendimiento promedio 

In [278]:
rets_mean = rets.mean()

##### Matriz de Correlación

In [279]:
corr = rets.corr()

#### Construcción de BenchMark

##### Rendimientos Diarios

In [280]:
rets_b = benchmark.pct_change().dropna()

##### Rendimiento Promedio 

In [281]:
rets_b_mean = rets_b.mean()

---

#### Mínima Semi-Varianza 

##### Scipy Library

In [282]:
scipy_semi_var = scipy_semivar(rets, corr)
scipy_semi_var

{'ABBV': 0.20027065338187994,
 'ISRG': 0.025472854448396448,
 'JNJ': 0.3955905769463641,
 'LLY': 0.1415935359861667,
 'MRK': 0.23707237923719288}

In [283]:
sum(scipy_semi_var.values())

1.0000000000000002

---

#### Ratio Omega 

##### Ratio Omega individual 

In [284]:
rets_below_zero = rets[rets < 0].fillna(0)
rets_above_zero=rets[rets > 0].fillna(0)
downside_risk=rets_below_zero.std()
upside_risk=rets_above_zero.std()
downside_risk = np.array(downside_risk)
upside_risk/downside_risk

Ticker
ABBV    0.900431
ISRG    1.053483
JNJ     1.044467
LLY     1.296354
MRK     0.942896
dtype: float64

##### Scipy Library

In [285]:
scipy_omega = scipy_omega(rets)
scipy_omega

{'ABBV': 5.608659069449639e-16,
 'ISRG': 3.191891195797325e-16,
 'JNJ': 1.5265566588595902e-16,
 'LLY': 0.9999999999999987,
 'MRK': 2.2097540209045966e-16}

In [286]:
sum(scipy_omega.values()).round(4)

1.0

---

#### Semi-Varianza Objetivo 

In [287]:
objetivo_weights = objetivo(rets, rets_b, corr)
objetivo_weights

{'ABBV': 0.147970019416079,
 'ISRG': 0.014375450465476364,
 'JNJ': 0.5734245173548107,
 'LLY': 0.04182210358814581,
 'MRK': 0.22240790917548806}

In [288]:
sum(objetivo_weights.values())

1.0

En esta función se puede observar que el activo con mayor peso en la minimización de la Semi-Varianza respecto al benchmark `VDC` es `ISRG`, lo que indica que este activo contribuye menos a las pérdidas grandes en comparación con otros, por lo que es una buena elección si buscamos un portafolio adverso al riesgo.

Por otro lado, el activo con menor peso en la minimización de la Semi-Varianza respecto al benchmark `VDC` es `JNJ`, lo que sugiere que este activo tiene mayor probabilidad de generar pérdidas frecuentes, y por eso recibe un peso menor en un portafolio que busca controlar el riesgo.



---

#### Conclusión 

Se puede concluir que la diferencia entre las teorías TMP (Teoría Moderna de Portafolio) y TPMP (Teoría Pos-Moderna de Portafolio) está en la forma en que se realizan los cálculos y en cómo se interpreta el riesgo. La TMP se basa únicamente en la varianza, sin distinguir si la volatilidad es positiva o negativa. Su ventaja es que es más sencilla de aplicar e interpretar, ya que con un solo cálculo se puede analizar el riesgo. Sin embargo, su principal desventaja es que trata por igual las ganancias y las pérdidas, lo cual no es ideal si se busca una inversión más segura o conservadora.

En cambio, la TPMP utiliza más métricas que permiten evaluar mejor el comportamiento de los activos, enfocándose especialmente en la volatilidad negativa . Esto hace que sus resultados sean más realistas y útiles para inversionistas que buscan proteger su capital. No obstante, su aplicación es más compleja tanto matemática como computacionalmente, lo cual representa su mayor desventaja.

Al elegir entre estas teorías es muy importante conocerr  y tener en claro qué tipo de riesgo estamos dispuestos a asumir en nuestra inversión, ya que cada enfoque responde a necesidades diferentes.

En nuestro caso, la Teoría Posmoderna de Portafolio fue considerablemente más difícil de aplicar, ya que resultó muy demandante a nivel computacional. Hubo momentos en los que Jupyter Notebook se llegaba a trabar debido al alto consumo de recursos, lo que dificultó el trabajo. Sin embargo, a pesar de estas complicaciones técnicas, logramos obtener ponderaciones adecuadas en cada una de las funciones implementadas, lo que permitió alcanzar resultados coherentes con los objetivos del modelo.
